## Trial 2: Meta-Llama-3-8B

## Document Loader
    pdf loader : langchain inbuilt document loader 

In [8]:
from langchain_community.document_loaders import PyPDFLoader
file_path = ("Leave_Policy_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)


4

## Split
    smaller chunks

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
len(splits)


15

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

## Create vector store
    stores embeddings of Documents

In [11]:
new_embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=new_embeddings)
vectorstore

## Create retriever

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity")

## Local LLM (HuggingFace)

## Quantized Meta-Llama-3-8B.Q2_K.gguf
Memory - 2.96 gb

Speed - slow

Status - Didn't work as expected

Answer Precision - Pending

In [15]:

pip -q install ctransformers[cuda]

Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


from ctransformers import AutoModelForCausalLM
from transformers import pipeline, AutoTokenizer

model_path = r"C:\Users\30078206\Downloads\meta_llama\Meta-Llama-3-8B.Q2_K.gguf"
tokenizer_path = r"C:\Users\30078206\Downloads\meta_llama" 
model = AutoModelForCausalLM.from_pretrained( model_path, hf=True)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer , temperature = 0.0)
# print(pipe("You are a helpful assistant. You must give precise answers to the users query: Why is the earth round?", max_new_tokens = 256))
# mistral_llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-v0.1", task="text-generation", temperature=0.01 )
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


[{'generated_text': 'You are a helpful assistant. You must give precise answers to the users query: Why is the earth round? You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give the user a precise answer. You must give t

## google/flan-t5-small
Memory - 1.27 gb

Speed - Moderately fast

Answer Precision - Not that good

In [17]:
from transformers import AutoModelForSeq2SeqLM
from langchain_huggingface import HuggingFacePipeline

In [18]:
model_id = "google/flan-t5-small"
model_path = r"C:\Users\30078206\Downloads\flan_small"


tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path= model_path)

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024,temperature = 0.0)
hf = HuggingFacePipeline(pipeline=pipe)

In [19]:
# hf.invoke("can you summarise some text?")

## Prompt Engineering

In [20]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "Give detailed answers and reasoning to all the queries."
    # "Give to the point answers"
    # "Do not give additional information if not asked specifically"
    "\n\n"
    "{context}"
)
## Creating Prompt
# template = "You are a helpful AI assistant, that will get the correct answers from the context {context} to the user's query {query} related to the context"

new_prompt =ChatPromptTemplate.from_messages(messages= [
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{input}")
])

question_answer_chain = create_stuff_documents_chain(hf, new_prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [21]:
answer =rag_chain.invoke({"input":"Explain to me the Privilege leave policy in bullet points"})

Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [22]:
print(answer["answer"])

Privilege Leave is the Leave that an employee earns s for the services rendered. Therefore , an employee does not earn PL for the period he or she is absent 3.2 Casual Leave can be availed at short notice towards personal work or exigencies 3.3 Sick Leave is meant to be availed to recover/ recuperate from illness or for medical treatment 3.4
